In [ ]:
import torch
from torch import nn

In [ ]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super().__init__()
        self.conv_block = nn.Sequential(nn.Conv2d(in_channels, out_channels, bias=False, **kwargs),
                                        nn.BatchNorm2d(out_channels, eps=0.001),
                                        nn.ReLU())
    def forward(self, x):
        x = self.conv_block(x)
        return x

class Inception_Stem(nn.Module):
    # Figure 3.
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(BasicConv2d(3, 32, kernel_size=3, stride=2),
                                   BasicConv2d(32, 32, kernel_size=3),
                                   BasicConv2d(32, 64, kernel_size=3, padding=1))
        self.branch3x3_pool = nn.MaxPool2d(3, stride=2)
        self.branch3x3_conv = BasicConv2d(64, 96, kernel_size=3, stride=2)

        self.branch7x7a = nn.Sequential(BasicConv2d(160, 64, kernel_size=1),
                                        BasicConv2d(64, 96, kernel_size=3))
        self.branch7x7b = nn.Sequential(BasicConv2d(160, 64, kernel_size=1),
                                        BasicConv2d(64, 64, kernel_size=(7, 1), padding=(3, 0)),
                                        BasicConv2d(64, 64, kernel_size=(1, 7), padding=(0, 3)),
                                        BasicConv2d(64, 96, kernel_size=3))

        self.branchpoola = BasicConv2d(192, 192, kernel_size=3, stride=2)
        self.branchpoolb = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = [self.branch3x3_pool(x),
             self.branch3x3_conv(x)]
        x = torch.cat(x, 1)

        x = [self.branch7x7a(x),
             self.branch7x7b(x)]
        x = torch.cat(x, 1)

        x = [self.branchpoola(x),
             self.branchpoolb(x)]
        x = torch.cat(x, 1)
        return x

class InceptionA(nn.Module):
    # Figure 4.
    def __init__(self, input_channels):
        super().__init__()

        self.branchpool = nn.Sequential(nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                                        BasicConv2d(input_channels, 96, kernel_size=1))

        self.branch1x1 = BasicConv2d(input_channels, 96, kernel_size=1)

        self.branch3x3 = nn.Sequential(BasicConv2d(input_channels, 64, kernel_size=1),
                                       BasicConv2d(64, 96, kernel_size=3, padding=1))

        self.branch3x3dbl = nn.Sequential(BasicConv2d(input_channels, 64, kernel_size=1),
                                            BasicConv2d(64, 96, kernel_size=3, padding=1),
                                            BasicConv2d(96, 96, kernel_size=3, padding=1))

    def forward(self, x):
        x = [self.branchpool(x), self.branch1x1(x), self.branch3x3(x), self.branch3x3dbl(x)]
        return torch.cat(x, 1)

class ReductionA(nn.Module):
    # Figure 7.
    # The k, l, m, n numbers represent filter bank sizes which can be looked up in Table 1.
    def __init__(self, input_channels, k, l, m, n):
        super().__init__()

        self.branchpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.branch3x3 = BasicConv2d(input_channels, n, kernel_size=3, stride=2)
        self.branch3x3dbl = nn.Sequential(BasicConv2d(input_channels, k, kernel_size=1),
                                          BasicConv2d(k, l, kernel_size=3, padding=1),
                                          BasicConv2d(l, m, kernel_size=3, stride=2))

    def forward(self, x):
        x = [self.branchpool(x), self.branch3x3(x), self.branch3x3dbl(x)]
        return torch.cat(x, 1)

class InceptionB(nn.Module):

    # Figure 5.
    def __init__(self, input_channels):
        super().__init__()

        self.branchpool = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1),
            BasicConv2d(input_channels, 128, kernel_size=1))

        self.branch1x1 = BasicConv2d(input_channels, 384, kernel_size=1)

        self.branch7x7 = nn.Sequential(
            BasicConv2d(input_channels, 192, kernel_size=1),
            BasicConv2d(192, 224, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(224, 256, kernel_size=(7, 1), padding=(3, 0))) # 논문은 여기도 1x7로 되어있음. 하지만 오류일 것으로 추정

        self.branch7x7dbl = nn.Sequential(
            BasicConv2d(input_channels, 192, kernel_size=1),
            BasicConv2d(192, 192, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(192, 224, kernel_size=(7, 1), padding=(3, 0)),
            BasicConv2d(224, 224, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(224, 256, kernel_size=(7, 1), padding=(3, 0)))

    def forward(self, x):
        x = [self.branchpool(x), self.branch1x1(x), self.branch7x7(x), self.branch7x7dbl(x)]
        return torch.cat(x, 1)

class ReductionB(nn.Module):
    # Figure 8.
    def __init__(self, input_channels):
        super().__init__()

        self.branchpool = nn.MaxPool2d(kernel_size=3, stride=2)

        self.branch3x3 = nn.Sequential(
            BasicConv2d(input_channels, 192, kernel_size=1),
            BasicConv2d(192, 192, kernel_size=3, stride=2))

        self.branch7x7 = nn.Sequential(
            BasicConv2d(input_channels, 256, kernel_size=1),
            BasicConv2d(256, 256, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(256, 320, kernel_size=(7, 1), padding=(3, 0)),
            BasicConv2d(320, 320, kernel_size=3, stride=2))

    def forward(self, x):
        x = [self.branchpool(x), self.branch3x3(x), self.branch7x7(x)]
        return torch.cat(x, 1)

class InceptionC(nn.Module):
    # Figure 6.
    def __init__(self, input_channels):
        super().__init__()

        self.branchpool = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            BasicConv2d(input_channels, 256, kernel_size=1))

        self.branch1x1 = BasicConv2d(input_channels, 256, kernel_size=1)

        self.branch3x3 = BasicConv2d(input_channels, 384, kernel_size=1)
        self.branch3x3a = BasicConv2d(384, 256, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3b = BasicConv2d(384, 256, kernel_size=(3, 1), padding=(1, 0))

        self.branch3x3dbl = nn.Sequential(
            BasicConv2d(input_channels, 384, kernel_size=1),
            BasicConv2d(384, 448, kernel_size=(1, 3), padding=(0, 1)),
            BasicConv2d(448, 512, kernel_size=(3, 1), padding=(1, 0)))
        self.branch3x3dbla = BasicConv2d(512, 256, kernel_size=(3, 1), padding=(1, 0))
        self.branch3x3dblb = BasicConv2d(512, 256, kernel_size=(1, 3), padding=(0, 1))

    def forward(self, x):
        branchpool = self.branchpool(x)

        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3(x)
        branch3x3 = [self.branch3x3a(branch3x3),
                     self.branch3x3b(branch3x3)]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl(x)
        branch3x3dbl = [self.branch3x3dbla(branch3x3dbl),
                        self.branch3x3dblb(branch3x3dbl)]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branchpool]

        return torch.cat(outputs, 1)

class InceptionV4(nn.Module):
    # Figure 9.
    def __init__(self, A, B, C, k=192, l=224, m=256, n=384, class_nums=1000):
        super().__init__()
        self.stem = Inception_Stem()
        self.inception_a = nn.Sequential(*[InceptionA(384) for _ in range(A)])
        self.reduction_a = ReductionA(384, k, l, m, n)
        self.inception_b = nn.Sequential(*[InceptionB(1024) for _ in range(B)])
        self.reduction_b = ReductionB(1024)
        self.inception_c = nn.Sequential(*[InceptionC(1536) for _ in range(C)])
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout2d(0.2)
        self.linear = nn.Linear(1536, class_nums)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_a(x)
        x = self.reduction_a(x)
        x = self.inception_b(x)
        x = self.reduction_b(x)
        x = self.inception_c(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        x = torch.flatten(x,1)
        x = self.linear(x)
        return x

In [ ]:
model = InceptionV4(4, 7, 3)
# print(model)
!pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,299,299), device='cpu')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/usr/local/lib/python3.9/dist-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                             Output Shape              Param #
InceptionV4                                        [2, 1000]                 --
├─Inception_Stem: 1-1                              [2, 384, 35, 35]          --
│    └─Sequential: 2-1                             [2, 64, 147, 147]         --
│    │    └─BasicConv2d: 3-1                       [2, 32, 149, 149]         928
│    │    └─BasicConv2d: 3-2                       [2, 32, 147, 147]         9,280
│    │    └─BasicConv2d: 3-3                       [2, 64, 147, 147]         18,560
│    └─MaxPool2d: 2-2                              [2, 64, 73, 73]           --
│    └─BasicConv2d: 2-3                            [2, 96, 73, 73]           --
│    │    └─Sequential: 3-4                        [2, 96, 73, 73]           55,488
│    └─Sequential: 2-4                             [2, 96, 71, 71]           --
│    │    └─BasicConv2d: 3-5                       [2, 64, 73, 73]           10,368
│    │    └─BasicCo

In [ ]:
x = model(torch.randn(2,3,299,299))
print(x.shape)

torch.Size([2, 1000])


In [ ]:
class InceptionResNetA(nn.Module):
    # Figure 16.
    def __init__(self, input_channels):
        super().__init__()

        self.branch1x1 = BasicConv2d(input_channels, 32, kernel_size=1)
        self.branch3x3 = nn.Sequential(
            BasicConv2d(input_channels, 32, kernel_size=1),
            BasicConv2d(32, 32, kernel_size=3, padding=1))
        self.branch3x3dbl = nn.Sequential(
            BasicConv2d(input_channels, 32, kernel_size=1),
            BasicConv2d(32, 48, kernel_size=3, padding=1),
            BasicConv2d(48, 64, kernel_size=3, padding=1))
        self.reduction1x1 = nn.Conv2d(128, 384, kernel_size=1)
        self.bn = nn.BatchNorm2d(384)

        self.shortcut = nn.Sequential(nn.Conv2d(input_channels, 384, kernel_size=1),
                                      nn.BatchNorm2d(384))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = [self.branch1x1(x),
                    self.branch3x3(x),
                    self.branch3x3dbl(x)]
        residual = torch.cat(residual, 1)
        residual = self.reduction1x1(residual) * 0.1 # Figure 20. 에 맞게 이 모듈에만 0.1이 없길래 추가함
        residual = self.bn(residual)

        shortcut = self.shortcut(x)

        output = self.relu(residual + shortcut)
        return output

class InceptionResNetReductionA(nn.Module):
    # Figure 7.
    # The k, l, m, n numbers represent filter bank sizes which can be looked up in Table 1.
    def __init__(self, input_channels, k, l, m, n):
        super().__init__()

        self.branchpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.branch3x3 = BasicConv2d(input_channels, n, kernel_size=3, stride=2)
        self.branch3x3dbl = nn.Sequential(BasicConv2d(input_channels, k, kernel_size=1),
                                          BasicConv2d(k, l, kernel_size=3, padding=1),
                                          BasicConv2d(l, m, kernel_size=3, stride=2))

    def forward(self, x):
        x = [self.branchpool(x), self.branch3x3(x), self.branch3x3dbl(x)]
        return torch.cat(x, 1)

class InceptionResNetB(nn.Module):
    # Figure 17.
    def __init__(self, input_channels):
        super().__init__()

        self.branch1x1 = BasicConv2d(input_channels, 192, kernel_size=1)
        self.branch7x7 = nn.Sequential(
            BasicConv2d(input_channels, 128, kernel_size=1),
            BasicConv2d(128, 160, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(160, 192, kernel_size=(7, 1), padding=(3, 0)))
        self.reduction1x1 = nn.Conv2d(384, 1154, kernel_size=1)
        self.bn = nn.BatchNorm2d(1154)

        self.shortcut = nn.Sequential(nn.Conv2d(input_channels, 1154, kernel_size=1),
                                      nn.BatchNorm2d(1154))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = [self.branch1x1(x),
                    self.branch7x7(x)]
        residual = torch.cat(residual, 1)
        residual = self.reduction1x1(residual) * 0.1 # Figure 20.
        residual = self.bn(residual)

        shortcut = self.shortcut(x)

        output = self.relu(residual + shortcut)
        return output

class InceptionResNetReductionB(nn.Module):
    # Figure 18.
    def __init__(self, input_channels):
        super().__init__()

        self.branchpool = nn.MaxPool2d(3, stride=2)

        self.branch3x3a = nn.Sequential(
            BasicConv2d(input_channels, 256, kernel_size=1),
            BasicConv2d(256, 384, kernel_size=3, stride=2))

        self.branch3x3b = nn.Sequential(
            BasicConv2d(input_channels, 256, kernel_size=1),
            BasicConv2d(256, 288, kernel_size=3, stride=2))

        self.branch3x3dbl = nn.Sequential(
            BasicConv2d(input_channels, 256, kernel_size=1),
            BasicConv2d(256, 288, kernel_size=3, padding=1),
            BasicConv2d(288, 320, kernel_size=3, stride=2))

    def forward(self, x):
        x = [self.branch3x3a(x), self.branch3x3b(x), self.branch3x3dbl(x), self.branchpool(x)]
        return torch.cat(x, 1)

class InceptionResNetC(nn.Module):
    # Figure 19.
    def __init__(self, input_channels):
        super().__init__()

        self.branch1x1 = BasicConv2d(input_channels, 192, kernel_size=1)
        self.branch3x3 = nn.Sequential(
            BasicConv2d(input_channels, 192, kernel_size=1),
            BasicConv2d(192, 224, kernel_size=(1, 3), padding=(0, 1)),
            BasicConv2d(224, 256, kernel_size=(3, 1), padding=(1, 0)))
        self.reduction1x1 = nn.Conv2d(448, 2048, kernel_size=1)
        self.bn = nn.BatchNorm2d(2048)

        self.shortcut = nn.Sequential(nn.Conv2d(input_channels, 2048, kernel_size=1),
                                      nn.BatchNorm2d(2048))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = [self.branch1x1(x),
                    self.branch3x3(x)]
        residual = torch.cat(residual, 1)
        residual = self.reduction1x1(residual) * 0.1 # Figure 20.
        residual = self.bn(residual)

        shortcut = self.shortcut(x)

        output = self.relu(residual + shortcut)
        return output

class InceptionResNetV2(nn.Module):
    # Figure 15.
    def __init__(self, A, B, C, k=256, l=256, m=384, n=384, class_nums=1000):
        super().__init__()
        self.stem = Inception_Stem()
        self.inception_resnet_a = nn.Sequential(*[InceptionResNetA(384) for _ in range(A)])
        self.reduction_a = InceptionResNetReductionA(384, k, l, m, n)
        self.inception_resnet_b = nn.Sequential(InceptionResNetB(1152),
                                                *[InceptionResNetB(1154) for _ in range(B-1)])
        self.reduction_b = InceptionResNetReductionB(1154)
        self.inception_resnet_c = nn.Sequential(InceptionResNetC(2146),
                                                *[InceptionResNetC(2048) for _ in range(C-1)])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # Dropout (keep 0.8)
        self.dropout = nn.Dropout2d(0.2)
        self.linear = nn.Linear(2048, class_nums)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_resnet_a(x)
        x = self.reduction_a(x)
        x = self.inception_resnet_b(x)
        x = self.reduction_b(x)
        x = self.inception_resnet_c(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        x = torch.flatten(x,1)
        x = self.linear(x)
        return x

In [ ]:
model = InceptionResNetV2(5, 10, 5)
# print(model)
summary(model, input_size=(2,3,299,299), device='cpu')

Layer (type:depth-idx)                             Output Shape              Param #
InceptionResNetV2                                  [2, 1000]                 --
├─Inception_Stem: 1-1                              [2, 384, 35, 35]          --
│    └─Sequential: 2-1                             [2, 64, 147, 147]         --
│    │    └─BasicConv2d: 3-1                       [2, 32, 149, 149]         928
│    │    └─BasicConv2d: 3-2                       [2, 32, 147, 147]         9,280
│    │    └─BasicConv2d: 3-3                       [2, 64, 147, 147]         18,560
│    └─MaxPool2d: 2-2                              [2, 64, 73, 73]           --
│    └─BasicConv2d: 2-3                            [2, 96, 73, 73]           --
│    │    └─Sequential: 3-4                        [2, 96, 73, 73]           55,488
│    └─Sequential: 2-4                             [2, 96, 71, 71]           --
│    │    └─BasicConv2d: 3-5                       [2, 64, 73, 73]           10,368
│    │    └─BasicCo

In [ ]:
x = torch.randn(2,3,299,299)
print(model(x).shape)

torch.Size([2, 1000])
